Know where you're running python from

In [1]:
import sys, os
print(sys.executable)

C:\Users\kople\anaconda3\python.exe


In [2]:
print(sys.path)

['C:\\Users\\kople\\Documents\\Work\\RA\\Student-Engagement_Prediction\\notebooks\\data_prep', 'C:\\Users\\kople\\anaconda3\\python39.zip', 'C:\\Users\\kople\\anaconda3\\DLLs', 'C:\\Users\\kople\\anaconda3\\lib', 'C:\\Users\\kople\\anaconda3', '', 'C:\\Users\\kople\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Users\\kople\\anaconda3\\lib\\site-packages', 'C:\\Users\\kople\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\kople\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\kople\\anaconda3\\lib\\site-packages\\Pythonwin']


In [3]:
os.getcwd()

'C:\\Users\\kople\\Documents\\Work\\RA\\Student-Engagement_Prediction\\notebooks\\data_prep'

## Package Imports

In [4]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import kaleido
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
from tqdm.notebook import tqdm_notebook,trange
from tqdm import tqdm
from time import sleep
from functools import reduce

In [5]:
ppath = 'C:/Users/kople/Documents/Work/RA/data/processed/sess1920sem1/'
path = 'C:/Users/kople/Documents/Work/RA/data/raw/sess1920sem1/'
plots_path = 'C:/Users/kople/Documents/Work\RA/Student-Engagement_Prediction/plots'

## Data imports

In [7]:
assignments = pd.read_csv(f'{ppath}assignment_consolidated.csv')
courses = pd.read_csv(f'{ppath}course_consolidated.csv')
grades_courses = pd.read_csv(f'{ppath}grades_courses.csv')
grades_consolidated = pd.read_csv(f'{ppath}grades_consolidated.csv')
quiz_merged = pd.read_csv(f'{ppath}quiz_merged.csv')
forumposts = pd.read_csv(f'{ppath}forumposts.csv')
students = pd.read_csv(f'{ppath}students.csv')
student_log = pd.read_csv(f'{ppath}student_log.csv')
student_logins = pd.read_csv(f'{ppath}student_logins.csv')
teachers = pd.read_csv(f'{ppath}teachers.csv')

# 'per user' df's
assignment_per_user = pd.read_csv(f'{ppath}assignment_per_user.csv')
attendance_sessions_per_student = pd.read_csv(f'{ppath}attendance_sessions_per_student.csv')
courses_per_user = pd.read_csv(f'{ppath}courses_per_user.csv')
log_actions_per_user = pd.read_csv(f'{ppath}log_actions_per_user.csv')
mean_posts_per_user = pd.read_csv(f'{ppath}mean_posts_per_user.csv')
posts_per_discussion_per_user = pd.read_csv(f'{ppath}posts_per_discussion_per_user.csv')
question_attempt_state_count = pd.read_csv(f'{ppath}question_attempt_state_count.csv')
questionattempts_per_user = pd.read_csv(f'{ppath}questionattempts_per_user.csv')
questionnaire_attempts = pd.read_csv(f'{ppath}questionnaire_attempts.csv')
questionnaire_attempts_stats = pd.read_csv(f'{ppath}questionnaire_attempts_stats.csv')
quiz_count_quizgrades = pd.read_csv(f'{ppath}quiz_count_quizgrades.csv')
quiz_count_quizmerged = pd.read_csv(f'{ppath}quiz_count_quizmerged.csv')
quiz_latest_attempts = pd.read_csv(f'{ppath}quiz_latest_attempts.csv')
quiz_per_user_quizattempts = pd.read_csv(f'{ppath}quiz_per_user_quizattempts.csv')
login_count = pd.read_csv(f'{ppath}login_count.csv')
quizattempt_stats = pd.read_csv(f'{ppath}quizattempt_stats.csv')
survey_per_user = pd.read_csv(f'{ppath}survey_per_user.csv')
surveyquestion_per_user = pd.read_csv(f'{ppath}surveyquestion_per_user.csv')
total_posts_per_user = pd.read_csv(f'{ppath}total_posts_per_user.csv')
user_per_discussion = pd.read_csv(f'{ppath}users_per_discussion.csv')


C:\Users\kople\AppData\Local\Temp\ipykernel_19372\3566694949.py:7: DtypeWarning: Columns (14,15,16,17,18,20,29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  students = pd.read_csv(f'{ppath}students.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/kople/Documents/Work/RA/data/processed/sess1920sem1/student_log.csv'

In [ ]:
frames = [assignment_per_user,
          attendance_sessions_per_student,
          courses_per_user,
          log_actions_per_user,
          mean_posts_per_user,
          question_attempt_state_count,
          questionattempts_per_user,
          questionnaire_attempts,
          questionnaire_attempts_stats,
          quiz_count_quizgrades,
          quiz_count_quizmerged,
          quiz_latest_attempts,
          login_count,
          quizattempt_stats,
          survey_per_user,
          surveyquestion_per_user,
          total_posts_per_user]

## Custom Functions

In [ ]:
def items_per_user(df,feature, parent_feature=None):
    """For a given dataframe and feature within that dataframe, items_per_user() returns
    a dataframe with the number of items in the feature that belong to or are attributed
    to the user id. Note that the function assumes that the df has a column 'userid'."""

    #preprocessing: sort the df by userid
    users = df.sort_values(by='userid').userid.unique()
    n_items_per_user = []
    if feature not in df.columns:
        print('The column', feature,' does not exist')
        pass
    if parent_feature==None:

        for user in users:
            #select portion of dataframe specific to the single user and then count number of unique values for the feature
            n_items=df[df.userid==user][feature].nunique()
            #add the data to the array
            n_items_per_user.append(n_items)
            #create the dataframe
        feature = 'number_of_' + feature +'s' #changing the name of the item column
        items_per_user_df = pd.DataFrame({'userid':users,feature:n_items_per_user})

    # else:
    #     # insert each item of the parent feature into a list
    #     parents = df[parent_feature].sort_values().unique()
    #     users = df.sort_values(by=parent_feature).userid.unique()
    #
    #     for user in users: # selecting a user
    #         for parent in parents: # iterating through all the parent_feature items
    #             n_items = df[(df.userid==user)&(df[parent_feature]==parent)][feature].nunique()
    #             n_items_per_user.append(n_items)
    #     # create the dataframe
    #     feature = 'number_of_' + feature +'s' #changing the name of the item column
    #     parent_feature = parent_feature+'id'
    #     items_per_user_df = pd.DataFrame({'userid':users, parent_feature:parent, feature:n_items_per_user})



    #
    # for item in items_parent_feature:
    #     # select portion of dataframe specific to the item, count the number of items in the child feature and put into list
    #     n_items = df[df[parent_feature]==item][feature].nunique()
    #     users = df[df[parent_feature]==item].userid.unique()
    #     n_items_per_user.append(n_items)

    return items_per_user_df


def pcnt_df(df,feature,lbound=None,hbound=None):
    """Calculate the portion(in percentage) of the dataframe that is populated by values bound by lbound and hbound (You can also use only one bound)"""
    #preprocessing
    df_notna = df[df[feature].notna()] #select portion of dataframe that is not empty

    if feature not in df.columns:
        raise Exception(feature, "is not a column of the dataframe")
    #if x not in df[feature]:
    #raise Exception(x, "is not within the range of the selected feature.")
    else:
        if hbound is None:
            df_above_lbound = df_notna[df_notna[feature]>lbound]
            pcnt = df_above_lbound.shape[0]/df_notna.shape[0]

        elif lbound is None:
            df_below_hbound = df_notna[df_notna[feature]<hbound]
            pcnt = df_below_hbound.shape[0]/df_notna.shape[0]

        elif lbound is not None and hbound is not None:
            df_btn_l_h = df_notna[(df_notna[feature]>lbound)&(df_notna[feature]<hbound)]
            pcnt = df_btn_l_h.shape[0]/df_notna.shape[0]

        return pcnt*100

# get name of df
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

# find statistical max,min and mean for the data on attempts
def find_stats(df,attempt_column):
    stat_all = []
    for user in df.userid.unique():
        df_user = df.loc[df.userid==user]
        max_attempt = df_user[attempt_column].max()
        min_attempt = df_user[attempt_column].min()
        mean_attempt = df_user[attempt_column].mean()
        stat_user = [max_attempt,min_attempt,mean_attempt]
        stat_all.append(stat_user)

    max_attempts = [el[0] for el in stat_all]
    min_attempts = [el[1] for el in stat_all]
    mean_attempts = [el[2] for el in stat_all]

    df2 = pd.DataFrame()
    df2['userid'] = df.userid.unique()
    df2[f'max_{attempt_column}'] = max_attempts
    df2[f'min_{attempt_column}'] = min_attempts
    df2[f'mean_{attempt_column}'] = mean_attempts

    return df2

## EDA: Plots

### assignment count

In [ ]:
assignment_per_user

In [ ]:
print(assignment_per_user.columns)
fig = px.scatter(assignment_per_user,
                 x='userid',
                 y='number_of_assignments',
                 color='number_of_assignments')
fig.show()

In [ ]:
fig = px.box(assignment_per_user,
             y='number_of_assignments')
fig.show()

In [ ]:
fig = px.histogram(assignment_per_user,
                   x='number_of_assignments',
                   color='number_of_assignments')
fig.show()

### attendance sessions

In [ ]:
attendance_sessions_per_student.head()

In [ ]:
fig = px.box(attendance_sessions_per_student,
             y='number_of_sessionids')
fig.show()

In [ ]:
fig = px.histogram(attendance_sessions_per_student,
                   x='number_of_sessionids')
fig.show()

### course count

In [ ]:
courses_per_user.head()

In [ ]:
fig = px.scatter(courses_per_user,
                 x='userid',
                 y='number_of_courseids')
fig.show()

In [ ]:
fig = px.box(courses_per_user,
                 y='number_of_courseids')
fig.show()

In [ ]:
fig = px.histogram(courses_per_user,
                 x='number_of_courseids',
                 color='number_of_courseids')
fig.show()

### login count

In [ ]:
login_count.head()

In [ ]:
fig = px.scatter(login_count,
                 x='userid',
                 y='number_of_logins',
                 color='number_of_logins')
fig.show()

In [ ]:
fig = px.box(login_count,
             y='number_of_logins')
fig.show()

In [ ]:
fig = px.histogram(login_count,
                   x='number_of_logins',
                   color='number_of_logins')
fig.show()

### Log actions

In [ ]:
log_actions_per_user

In [ ]:
log_actions_per_user.describe().loc['mean',:]

### mean posts per user

In [ ]:
mean_posts_per_user

In [ ]:
fig = px.scatter(mean_posts_per_user,
                 x='userid',
                 y='mean_posts')
fig.show()

In [ ]:
fig = px.histogram(mean_posts_per_user,x='mean_posts')
fig.show()

In [ ]:
fig = px.box(mean_posts_per_user,y='mean_posts')
fig.show()

### question attempt state count

In [ ]:
question_attempt_state_count

In [ ]:
msno.matrix(question_attempt_state_count)

### question attempts per user

In [ ]:
questionattempts_per_user

In [ ]:
questionattempts_per_user.rename(columns={'number_of_attempts':'question_attempts'},inplace=True)

In [ ]:
fig = px.scatter(questionattempts_per_user,x='userid',y='question_attempts')
fig.show()

In [ ]:
fig = px.box(questionattempts_per_user,y='question_attempts')
fig.show()

In [ ]:
fig = px.histogram(questionattempts_per_user,x='question_attempts')
fig.show()

In [ ]:
fig = px.histogram(questionattempts_per_user[questionattempts_per_user.question_attempts<1000],x='question_attempts')
fig.show()

### questionnaire attempts

In [ ]:
questionnaire_attempts

### questionnaire_attempt_stats

In [ ]:
questionnaire_attempts_stats.head()

In [ ]:
questionnaire_attempts_stats.drop(['userid'],axis=1).describe()

### Survey count

In [ ]:
survey_per_user

In [ ]:
fig = px.scatter(survey_per_user,
                 x='userid',
                 y='number_of_surveys',
                 color='number_of_surveys')
fig.show()

In [ ]:
fig = px.box(survey_per_user,
                 y='number_of_surveys')
fig.show()

In [ ]:
survey_per_user.describe()

### Survey question count

In [ ]:
surveyquestion_count = surveyquestion_per_user

In [ ]:
surveyquestion_count.head()

In [ ]:
fig = px.scatter(surveyquestion_count,
                 x='userid',
                 y='number_of_questions',
                 color='number_of_questions')
fig.show()

In [ ]:
fig = px.box(surveyquestion_count,
             y='number_of_questions')
fig.show()

In [ ]:
fig = px.histogram(surveyquestion_count,
                   x='number_of_questions',
                   color='number_of_questions')
fig.show()

### quiz count (quiz grades)

### quiz count (quiz merged)

In [ ]:
quiz_count_quizmerged

In [ ]:
fig = px.scatter(quiz_count_quizmerged,
                 x='userid',
                 y='number_of_quizs',
                 color='number_of_quizs')
fig.show()

In [ ]:
fig = px.box(quiz_count_quizmerged,
                 y='number_of_quizs')
fig.show()

In [ ]:
fig = px.histogram(quiz_count_quizmerged,
                   x='number_of_quizs',
                   color='number_of_quizs')
fig.show()

### quiz latest attempts (quiz attempts count)

In [ ]:
quiz_latest_attempts.sort_values('userid')

### quiz per user quiz (quiz attempts)

In [ ]:
quiz_per_user_quizattempts

In [ ]:
fig = px.scatter(quiz_per_user_quizattempts,
                 x='userid',
                 y='number_of_quizs',
                 color='number_of_quizs')
fig.show()

In [ ]:
fig = px.box(quiz_per_user_quizattempts,
             y='number_of_quizs')
fig.show()

In [ ]:
fig = px.histogram(quiz_per_user_quizattempts,
                   x='number_of_quizs',
                   color='number_of_quizs')
fig.show()

### Total posts

In [ ]:
total_posts_per_user.head()

In [ ]:
fig = px.scatter(total_posts_per_user,
                 x='userid',
                 y='count',
                 color='count')
fig.show()

In [ ]:
fig = px.box(total_posts_per_user,
                 y='count')
fig.show()

In [ ]:
fig = px.histogram(total_posts_per_user,
             x='count')
fig.show()

### User count in discussions

In [ ]:
user_count_discussion = user_per_discussion

In [ ]:
user_count_discussion

In [ ]:
fig = px.scatter(user_count_discussion,
                 x='discussion',
                 y='number_of_userids',
                 color='number_of_userids')
fig.show()

In [ ]:
fig = px.box(user_count_discussion,
             y='number_of_userids')
fig.show()

In [ ]:
fig = px.histogram(user_count_discussion,
                   x='number_of_userids')
fig.show()

## Feature Engineering

### Changing the column names of question_attempt_state_count

In [ ]:
question_attempt_state_count

In [ ]:
for column in question_attempt_state_count.columns:
    question_attempt_state_count.rename(columns={column:f'question_{column}'},inplace=True)

In [ ]:
question_attempt_state_count.columns

In [ ]:
question_attempt_state_count.rename(columns={'question_userid':'userid'},inplace=True)

In [ ]:
question_attempt_state_count.columns

### Changing Column names of questionnaire_attempts_stats

In [ ]:
questionnaire_attempts_stats

In [ ]:
questionnaire_attempts_stats =  questionnaire_attempts_stats.add_prefix('questionnaire_')

In [ ]:
questionnaire_attempts_stats.rename(columns={'questionnaire_userid':'userid'},inplace=True)

### Changing column name of quizattempt_stats

In [ ]:
quizattempt_stats

In [ ]:
quizattempt_stats =  quizattempt_stats.add_prefix('quiz_')
quizattempt_stats.rename(columns={'quiz_userid':'userid'},inplace=True)

In [ ]:
quizattempt_stats

### Changing column name of totalposts

In [ ]:
total_posts_per_user

In [ ]:
total_posts_per_user.rename(columns={'count':'total_posts'},inplace=True)

## The Final Dataframe

In [ ]:
frame_names = pd.DataFrame([get_df_name(frame) for frame in frames ],columns=['frames'])

In [ ]:
frame_names

finding out which dataframe has the highest number of userids

In [ ]:
max_userid = 1
for df in frames:
    if 'userid' in df.columns:
      print(f"{get_df_name(df)} has {df.userid.nunique()} userids")
      if df.userid.nunique()>max_userid:
          max_userid = df.userid.nunique()
          #print(f"{get_df_name(df)} has {df.userid.nunique()} userids")
print('-'*30)
print(max_userid)

Other than log, login_count has the most number of userid's. Therefore the final dataframe shall be prepared on top of this dataframe.

In [ ]:
login_count.head()

In [ ]:
students

In [ ]:
course_count = courses_per_user
assignment_count = assignment_per_user
survey_count = survey_per_user

In [ ]:
final = students.rename(columns={'id':'userid'}).merge(login_count,on='userid',how='left')
final = final.merge(attendance_sessions_per_student,on='userid',how='left')
final = final.merge(course_count,on='userid',how='left')
final = final.merge(assignment_count,on='userid',how='left')
final = final.merge(mean_posts_per_user,on='userid',how='left')
final = final.merge(total_posts_per_user,on='userid',how='left')
final = final.merge(question_attempt_state_count,on='userid',how='left')
final = final.merge(questionattempts_per_user,on='userid',how='left')
final = final.merge(questionnaire_attempts_stats,on='userid',how='left')
final = final.merge(quiz_per_user_quizattempts,on='userid',how='left')
final = final.merge(quizattempt_stats,on='userid',how='left')
final = final.merge(survey_count,on='userid',how='left')
final = final.merge(surveyquestion_count,on='userid',how='left')


In [ ]:
final.sort_values('userid',inplace=True)

In [ ]:
final

In [ ]:
msno.matrix(final)

In [ ]:
final.info()

In [ ]:
final.to_feather(f'{ppath}final.feather')

testing whether it worked

In [ ]:
final_load = pd.read_feather(f'{ppath}final.feather')

In [ ]:
final_load